In [2]:
# !pip install pytesseract
# !pip install nltk

In [1]:
import pandas as pd
import os
import pytesseract
from PIL import Image
import re

C:\Users\eswar\AppData\Local\Temp\ipykernel_8628\2788324195.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import nltk
nltk.download('words')


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [13]:
# setting parameters
std = 3

if std==3:
    user = "Madhu"
elif std==6:
    user = "bhavi"
else:
    user = "user"

sub = "science"
lesson = 1
file_name = str(std)+"_"+sub+"_LS"+str(lesson)

images_folder_path = 'scanned_files' 
ext_txt_file_path = file_name+'_01_extracted_text.txt'

In [4]:
# list all image files in the folder

def list_files_in_folder(images_folder_path):
    """Lists all files in the given folder."""
    try:
        # List all entries in the given folder
        entries = os.listdir(images_folder_path)
        files = [entry for entry in entries if os.path.isfile(os.path.join(images_folder_path, entry))]
        return files
    except FileNotFoundError:
        print("The folder does not exist.")
        return []
    except PermissionError:
        print("Permission denied.")
        return []

# Example usage
files = list_files_in_folder(images_folder_path)
print("Files in folder:", files)


Files in folder: ['20240302_085550.jpg', '20240302_085604.jpg', '20240302_085741.jpg', '20240302_085910.jpg', '20240302_090010.jpg', '20240302_090154.jpg', '20240302_090210.jpg', '20240302_090244.jpg', '20240302_090258.jpg', '20240302_090336.jpg', '20240302_090353.jpg', '20240302_090452.jpg', '20240302_090510.jpg', '20240302_090544.jpg', '20240302_090600.jpg', '20240302_090623.jpg', '20240302_090701.jpg', '20240302_090721.jpg', '20240302_090732.jpg', '20240302_090801.jpg', '20240302_090820.jpg']


In [5]:
# extract the text from images and save to extracted_text.txt
def extract_text_from_image(image_path):
    # Open the image file
    with Image.open(image_path) as img:
        # Use pytesseract to extract text from the image
        text = pytesseract.image_to_string(img)
        return text

def extract_text_from_list_of_image(files):
    pageno=1
    # Open the file in append mode
    with open(ext_txt_file_path, 'a') as data:
        # Iterate through a range or any iterable
        for file in files:
            image_path = images_folder_path+'/'+file
            page_no = f"\n\nPage nummber: {pageno}"+'\n'
            extracted_text = extract_text_from_image(image_path)
            # Append text to the file
            data.write(page_no+extracted_text)
            pageno+=1

extract_text_from_list_of_image(files)

In [6]:
# get unique words from extracted text
def get_unique_words(ext_txt_file_path):
    unique_words = set()

    # Open the file and read its contents
    with open(ext_txt_file_path, 'r') as file:
        # Read the text and split it into words
        words = file.read().split()

        # Add each word to the set of unique words
        for word in words:
            unique_words.add(word)

    return unique_words

# Example usage:

unique_words = get_unique_words(ext_txt_file_path)
unique_words_list = list(unique_words)
df = pd.DataFrame(unique_words_list)
df.columns=['unique_words']
df.to_csv(file_name+"_02_unique_words.csv",index=False)
df

,unique_words
0,Skla/
1,(Werm-up)°
2,"60,"
3,"117,"
4,is
...,...
2650,Let's
2651,Literacy
2652,F
2653,NekaritiPhens


In [7]:
# remove the words that are not in dictionary
import nltk
from nltk.corpus import words

# Download the word list
nltk.download('words')

# List of English words provided by nltk
english_vocab = set(w.lower() for w in words.words())

def filter_english_words(word_list):
    """Filter the given list of words, keeping only English dictionary words."""
    # Convert all words in the list to lower case
    word_list = [word.lower() for word in word_list]
    # Filter the list to keep only words found in the English vocabulary
    return [word for word in word_list if word in english_vocab]

# Example usage:
filtered_words = filter_english_words(unique_words_list)
df_dict_matched_words = pd.DataFrame(filtered_words)
df_dict_matched_words.columns=['dict_matched_words']
df_dict_matched_words.to_csv(file_name+"_03_dict_matched_words.csv", index=False)
df_dict_matched_words


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,dict_matched_words
0,is
1,dust
2,cut
3,hygiene
4,remember
...,...
1105,over
1106,large
1107,tha
1108,literacy


In [8]:
# Remove unwanted words that are manually notted
df_unwanted_words = pd.read_csv("remove_unwanted_words.csv")
df_unwanted_words_list = df_unwanted_words["unwanted_words"].to_list()

filtered_words = pd.read_csv(file_name+"_03_dict_matched_words.csv")
filtered_words = filtered_words["dict_matched_words"]
filtered_wanted_words = [word for word in filtered_words if word not in df_unwanted_words_list]
df_dict_matched_words = pd.DataFrame(filtered_wanted_words)
df_dict_matched_words.columns=['dict_matched_filtered_words']
df_dict_matched_words.to_csv(file_name+"_04_dict_matched_filtered_words.csv", index=False)
df_dict_matched_words

,dict_matched_filtered_words
0,is
1,dust
2,cut
3,hygiene
4,remember
...,...
988,decrease
989,roughage
990,over
991,large


In [9]:
# chunking
def chunk_around_word(sentence, target_word, sentence_len=10):
    cleaned_text = re.sub(r'\n', ' ', sentence)
    cleaned_text = re.sub(r'[^A-Za-z0-9 ]', '', cleaned_text)
    sentence_lower = cleaned_text.lower()
    target_word = target_word.lower()

    sentence_len_left = int(sentence_len/2)
    
    words = sentence_lower.split()
    target_index = words.index(target_word)  # Find the index of the target word

    # Calculate start and end indices for slicing the words list
    start_index = max(0, target_index - sentence_len_left)  # 5 words before
    end_index = min(len(words), target_index + sentence_len_left)  # 4 words after, +1 because of slice behavior

    # Join and return the chunk
    chunk = ' '.join(words[start_index:end_index])
    return chunk

# find sentance with the given word
def find_sentence_with_word(sentences, word):
    word = word.lower()  # Convert the word to lowercase
    sent = ""
    cleaned_sentence = ""
    for sentence in sentences:
        cleaned_text = re.sub(r'\n', ' ', sentence)
        cleaned_text = re.sub(r'[^A-Za-z0-9 ]', '', cleaned_text)
        if re.search(r'\b{}\b'.format(re.escape(word)), cleaned_text.lower()):
            sent = sentence
            cleaned_sentence = cleaned_text
            break
        else:
            sent = ""
            cleaned_sentence = ""

    return sent, cleaned_sentence
        

In [10]:
# sentance spliter

nltk.download('punkt')  # Download the required tokenizer data

def split_into_sentences(text):
    # Use the nltk tokenizer to split the text into sentences
    sentences = nltk.sent_tokenize(text)
    return sentences

# Read the text file
with open(ext_txt_file_path, "r", encoding="latin-1") as file:
    text = file.read()

# Split the text into sentences
sentences = split_into_sentences(text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# fill the sentence and chunk sentance columns
df = pd.read_csv(file_name+"_04_dict_matched_filtered_words.csv")

# Function to apply to each row
def find_and_chunk(row, sentences):
    word = row['dict_matched_filtered_words']
    sentence, cleaned_sentence = find_sentence_with_word(sentences, word)
    if sentence:
        chunked_sentence = chunk_around_word(sentence, word)
    else:
        chunked_sentence = None
    return pd.Series([sentence, cleaned_sentence, chunked_sentence])

# Apply the function to the DataFrame
df[['sentence', 'cleaned_sentence', 'chunked_sentence']] = df.apply(find_and_chunk, args=(sentences,), axis=1)
df.to_csv(file_name+"_05_words_with_chunked_sentence.csv", index=False)
df


,dict_matched_filtered_words,sentence,cleaned_sentence,chunked_sentence
0,is,Altura is hosted on MEE (Macmillan Education E...,Altura is hosted on MEE Macmillan Education Ev...,altura is hosted on mee macmillan
1,dust,Colour the things that need air\n(CG-1|06-2106...,Colour the things that need air CG10621064 i ...,find out that air contains dust particles ng c...
2,cut,ame Re\n= 3M Rp)\n~e et '\n\nAnimals such as c...,ame Re 3M Rp e et Animals such as cows and ...,and buffaloes first bite and cut leaves and gr...
3,hygiene,Hygiene: We must practice\ngood hygiene and ke...,Hygiene We must practice good hygiene and keep...,hygiene we must practice good
4,remember,"a Dea The brain helps us to see, hear, think, ...",a Dea The brain helps us to see hear think B E...,see hear think b ex remember and do many other
...,...,...,...,...
988,decrease,Guide them to research\n© create ewarenem abou...,Guide them to research create ewarenem about ...,ihe the reasons for the decrease in the poputa...
989,roughage,bo eaten | * Make mixed frult salad corract op...,bo eaten Make mixed frult salad corract opti...,corract option raw sabet about roughage compre...
990,over,The muscular system\n\nWhile making the toy ho...,The muscular system While making the toy hors...,horse we used coloured cloth over the framewor...
991,large,MEE is an online education hub that provides a...,MEE is an online education hub that provides a...,education hub that provides a large and indisp...


In [42]:
import pandas as pd
import nltk
from nltk.corpus import brown

# Ensure you've downloaded the necessary NLTK resources
nltk.download('brown')
nltk.download('punkt')

# Load all sentences from the Brown Corpus
brown_sentences = [' '.join(sent) for sent in brown.sents()]

def find_example_sentence(word):
    word = word.lower()
    for sentence in brown_sentences:
        if word in sentence.lower().split():
            return sentence
    return "No example sentence found."

def mask_word(word, sentence):
    word = word.lower()
    if re.search(r'\b{}\b'.format(re.escape(word)), sentence.lower()):
        masked_sentence = sentence.lower().replace(word, '*' * len(word))
    else:
        masked_sentence = sentence
    return masked_sentence.capitalize()  # Capitalize the first letter of the sentenc

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
df = pd.read_csv(file_name+"_05_words_with_chunked_sentence.csv")

# Add a column with the masked chunked sentence
df['masked_chunked_sentence'] = df.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['chunked_sentence']), axis=1)

#adding dummy columns openai_sentence,masked_openai_sentence
df['example_sentence'] = ""
df['masked_example_sentence'] = ""
df['openai_sentence'] = ""
df['masked_openai_sentence'] = ""
df.to_csv(file_name+"_06_words_with_chunked_eg_sentence.csv", index=False)
df.head()


,dict_matched_filtered_words,sentence,cleaned_sentence,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,is,Altura is hosted on MEE (Macmillan Education E...,Altura is hosted on MEE Macmillan Education Ev...,altura is hosted on mee macmillan,Altura ** hosted on mee macmillan,,,,
1,dust,Colour the things that need air\n(CG-1|06-2106...,Colour the things that need air CG10621064 i ...,find out that air contains dust particles ng c...,Find out that air contains **** particles ng c...,,,,
2,cut,ame Re\n= 3M Rp)\n~e et '\n\nAnimals such as c...,ame Re 3M Rp e et Animals such as cows and ...,and buffaloes first bite and cut leaves and gr...,And buffaloes first bite and *** leaves and gr...,,,,
3,hygiene,Hygiene: We must practice\ngood hygiene and ke...,Hygiene We must practice good hygiene and keep...,hygiene we must practice good,******* we must practice good,,,,
4,remember,"a Dea The brain helps us to see, hear, think, ...",a Dea The brain helps us to see hear think B E...,see hear think b ex remember and do many other,See hear think b ex ******** and do many other,,,,


In [21]:
# update openai sentences from master wordlist
df = pd.read_csv(file_name+"_06_words_with_chunked_eg_sentence.csv")
# Read the word_list_all CSV file into df1
df1 = pd.read_csv('word_list_all.csv')

# Merge df with df1
df_merged = pd.merge(df, df1, how='left', left_on='dict_matched_filtered_words', right_on='word')
df_merged.loc[df_merged['example_sentence_x'].isnull(), "example_sentence_x"]=df_merged.loc[df_merged['example_sentence_x'].isnull(), "example_sentence_y"]
df_merged.loc[df_merged['masked_example_sentence_x'].isnull(), "masked_example_sentence_x"]=df_merged.loc[df_merged['masked_example_sentence_x'].isnull(), "masked_example_sentence_y"]
df_merged.loc[df_merged['openai_sentence_x'].isnull(), "openai_sentence_x"]=df_merged.loc[df_merged['openai_sentence_x'].isnull(), "openai_sentence_y"]
df_merged.loc[df_merged['masked_openai_sentence_x'].isnull(), "masked_openai_sentence_x"]=df_merged.loc[df_merged['masked_openai_sentence_x'].isnull(), "masked_openai_sentence_y"]

df_merged.drop(['user','word','chunked_sentence_y','masked_chunked_sentence_y', 'example_sentence_y', 'masked_example_sentence_y', 'openai_sentence_y', 'masked_openai_sentence_y'], axis=1, inplace=True)
df_merged.rename(columns={'chunked_sentence_x':'chunked_sentence', 'masked_chunked_sentence_x':'masked_chunked_sentence', 'example_sentence_x':'example_sentence', 'masked_example_sentence_x':'masked_example_sentence', 'openai_sentence_x':'openai_sentence', 'masked_openai_sentence_x':'masked_openai_sentence'}, inplace=True)
df_merged['word_type']=file_name
df_merged=df_merged[['dict_matched_filtered_words', 'word_type', 'sentence', 'cleaned_sentence',
       'chunked_sentence', 'masked_chunked_sentence',
       'example_sentence', 'masked_example_sentence',
       'openai_sentence', 'masked_openai_sentence']]
df_merged.drop_duplicates(inplace=True,ignore_index=True)

C:\Users\eswar\AppData\Local\Temp\ipykernel_8628\525202710.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '["The City Purchasing Department , the jury said , `` is lacking in experienced clerical personnel as a result of city personnel policies '' ."
 "The City Purchasing Department , the jury said , `` is lacking in experienced clerical personnel as a result of city personnel policies '' ."
 'Moments later the V-1 skimmed across the landing strip , edging closer and closer to a touchdown -- then in a streamer of dust it landed .'
 ...
 'Cotten construed this as a veiled effort by Parkhouse to help Dallas and other large cities get money which Cotten felt could better be spent providing water for rural Texas .'
 'Cotten construed this as a veiled effort by Parkhouse to help Dallas and other large cities get money which Cotten felt could better be spent providing water for rural Texas .'
 'No example 

In [24]:
# Add a column with an example sentence from the Brown Corpus
df_merged.loc[df_merged['example_sentence'].isnull(), 'example_sentence']= df_merged.loc[df_merged['example_sentence'].isnull(), 'dict_matched_filtered_words'].apply(find_example_sentence)

# Add a column with the masked example sentence
df_merged['masked_example_sentence'] = df_merged.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['example_sentence']), axis=1)
df_merged.to_csv(file_name+"_10_merged_df_with_existing_openai_sentence.csv", index=False)
df_merged.head()

,dict_matched_filtered_words,word_type,sentence,cleaned_sentence,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,is,3_science_LS1,Altura is hosted on MEE (Macmillan Education E...,Altura is hosted on MEE Macmillan Education Ev...,altura is hosted on mee macmillan,Altura ** hosted on mee macmillan,"The City Purchasing Department , the jury said...","The city purchasing department , the jury said...",My cat is very fluffy.,My cat ** very fluffy.
1,dust,3_science_LS1,Colour the things that need air\n(CG-1|06-2106...,Colour the things that need air CG10621064 i ...,find out that air contains dust particles ng c...,Find out that air contains **** particles ng c...,Moments later the V-1 skimmed across the landi...,Moments later the v-1 skimmed across the landi...,Please dust the shelves.,Please **** the shelves.
2,cut,3_science_LS1,ame Re\n= 3M Rp)\n~e et '\n\nAnimals such as c...,ame Re 3M Rp e et Animals such as cows and ...,and buffaloes first bite and cut leaves and gr...,And buffaloes first bite and *** leaves and gr...,`` This is a very modest proposal cut to meet ...,`` this is a very modest proposal *** to meet ...,I cut paper with scissors.,I *** paper with scissors.
3,hygiene,3_science_LS1,Hygiene: We must practice\ngood hygiene and ke...,Hygiene We must practice good hygiene and keep...,hygiene we must practice good,******* we must practice good,"health , hygiene , medical aid and preventive ...","Health , ******* , medical aid and preventive ...",NaN,NaN
4,remember,3_science_LS1,"a Dea The brain helps us to see, hear, think, ...",a Dea The brain helps us to see hear think B E...,see hear think b ex remember and do many other,See hear think b ex ******** and do many other,"The outgoing members , whose four-year terms w...","The outgoing members , whose four-year terms w...",Remember to wash your hands.,******** to wash your hands.
...,...,...,...,...,...,...,...,...,...,...
814,broken,3_science_LS1,These complex substances\nhave to be broken do...,These complex substances have to be broken dow...,complex substances have to be broken down into...,Complex substances have to be ****** down into...,Just how many sub secrets were being handed ov...,Just how many sub secrets were being handed ov...,The vase is broken now.,The vase is ****** now.
815,decrease,3_science_LS1,Guide them to research\n© create ewarenem abou...,Guide them to research create ewarenem about ...,ihe the reasons for the decrease in the poputa...,Ihe the reasons for the ******** in the poputa...,"The volume of ADC cases will decrease , Martin...","The volume of adc cases will ******** , martin...",The temperature will decrease tonight.,The temperature will ******** tonight.
816,roughage,3_science_LS1,bo eaten | * Make mixed frult salad corract op...,bo eaten Make mixed frult salad corract opti...,corract option raw sabet about roughage compre...,Corract option raw sabet about ******** compre...,No example sentence found.,No example sentence found.,Eating roughage is good for digestion.,Eating ******** is good for digestion.
817,over,3_science_LS1,The muscular system\n\nWhile making the toy ho...,The muscular system While making the toy hors...,horse we used coloured cloth over the framewor...,Horse we used coloured cloth **** the framewor...,It permits the state to take over bank account...,It permits the state to take **** bank account...,The game is over.,The game is ****.


In [ ]:
# listing null values in openai sentences
df = pd.read_csv(file_name+"_06_words_with_chunked_eg_sentence.csv")
df = df_merged.loc[df_merged['openai_sentence'].isnull(),['dict_matched_filtered_words','openai_sentence']]
df.reset_index(inplace=True, drop=True)
df.to_csv(file_name+"_07_blank_openai_sentence.csv", index=False)
df

In [46]:
# Adding masked openai sentence to new words. 
# Manually enter the openai sentences using chatgpt and then use this funtion to fill masked sentance.
# Add a column with the masked example sentence
try:
    df = pd.read_csv(file_name+"_08_openai_sentence.csv")
    df['masked_openai_sentence'] = df.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['openai_sentence']), axis=1)
    df.to_csv(file_name+"_09_masked_openai_sentence.csv", index=False)
except:
    print("Manually enter the openai sentences using chatgpt and then use this funtion to fill masked sentance.")

df.head()

,dict_matched_filtered_words,openai_sentence,masked_openai_sentence
0,hygiene,Brushing teeth is good hygiene.,brushing teeth is good *******.
1,journey,We went on a long journey.,we went on a long *******.
2,initiative,Show initiative by starting first.,show ********** by starting first.
3,media,We watch TV on media.,we watch tv on *****.
4,conduct,Good conduct is always appreciated.,good ******* is always appreciated.


In [52]:
# merging the new openai sentances with existing sheet
df =  pd.read_csv(file_name+"_10_merged_df_with_existing_openai_sentence.csv")
df1 = pd.read_csv(file_name+"_09_masked_openai_sentence.csv")
df1.columns = ['dict_matched_filtered_words', 'openai_sentence_new', 'masked_openai_sentence_new']

df_merged = pd.merge(df, df1, how='left', on='dict_matched_filtered_words')
df_merged.loc[df_merged['openai_sentence'].isnull(), "openai_sentence"]=df_merged.loc[df_merged['openai_sentence'].isnull(), "openai_sentence_new"]
df_merged.loc[df_merged['masked_openai_sentence'].isnull(), "masked_openai_sentence"]=df_merged.loc[df_merged['masked_openai_sentence'].isnull(), "masked_openai_sentence_new"]
df_merged.drop(columns=['sentence', 'cleaned_sentence', 'openai_sentence_new','masked_openai_sentence_new'], inplace=True)
df_merged.rename(columns={'dict_matched_filtered_words':'word'}, inplace=True)

df_merged.to_csv(file_name+"_11_word_list.csv", index=False)
df_merged

,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,is,3_science_LS1,altura is hosted on mee macmillan,Altura ** hosted on mee macmillan,"The City Purchasing Department , the jury said...","The city purchasing department , the jury said...",My cat is very fluffy.,My cat ** very fluffy.
1,dust,3_science_LS1,find out that air contains dust particles ng c...,Find out that air contains **** particles ng c...,Moments later the V-1 skimmed across the landi...,Moments later the v-1 skimmed across the landi...,Please dust the shelves.,Please **** the shelves.
2,cut,3_science_LS1,and buffaloes first bite and cut leaves and gr...,And buffaloes first bite and *** leaves and gr...,`` This is a very modest proposal cut to meet ...,`` this is a very modest proposal *** to meet ...,I cut paper with scissors.,I *** paper with scissors.
3,hygiene,3_science_LS1,hygiene we must practice good,******* we must practice good,"health , hygiene , medical aid and preventive ...","Health , ******* , medical aid and preventive ...",Brushing teeth is good hygiene.,brushing teeth is good *******.
4,remember,3_science_LS1,see hear think b ex remember and do many other,See hear think b ex ******** and do many other,"The outgoing members , whose four-year terms w...","The outgoing members , whose four-year terms w...",Remember to wash your hands.,******** to wash your hands.
...,...,...,...,...,...,...,...,...
814,broken,3_science_LS1,complex substances have to be broken down into...,Complex substances have to be ****** down into...,Just how many sub secrets were being handed ov...,Just how many sub secrets were being handed ov...,The vase is broken now.,The vase is ****** now.
815,decrease,3_science_LS1,ihe the reasons for the decrease in the poputa...,Ihe the reasons for the ******** in the poputa...,"The volume of ADC cases will decrease , Martin...","The volume of adc cases will ******** , martin...",The temperature will decrease tonight.,The temperature will ******** tonight.
816,roughage,3_science_LS1,corract option raw sabet about roughage compre...,Corract option raw sabet about ******** compre...,No example sentence found.,No example sentence found.,Eating roughage is good for digestion.,Eating ******** is good for digestion.
817,over,3_science_LS1,horse we used coloured cloth over the framewor...,Horse we used coloured cloth **** the framewor...,It permits the state to take over bank account...,It permits the state to take **** bank account...,The game is over.,The game is ****.


In [29]:
df2


,user,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,is,3_science_LS1,altura is hosted on mee macmillan,Altura ** hosted on mee macmillan,"The City Purchasing Department , the jury said...","The city purchasing department , the jury said...",My cat is very fluffy.,My cat ** very fluffy.,Madhu
1,dust,3_science_LS1,find out that air contains dust particles ng c...,Find out that air contains **** particles ng c...,Moments later the V-1 skimmed across the landi...,Moments later the v-1 skimmed across the landi...,Please dust the shelves.,Please **** the shelves.,Madhu
2,cut,3_science_LS1,and buffaloes first bite and cut leaves and gr...,And buffaloes first bite and *** leaves and gr...,`` This is a very modest proposal cut to meet ...,`` this is a very modest proposal *** to meet ...,I cut paper with scissors.,I *** paper with scissors.,Madhu
3,hygiene,3_science_LS1,hygiene we must practice good,******* we must practice good,"health , hygiene , medical aid and preventive ...","Health , ******* , medical aid and preventive ...",Brushing teeth is good hygiene.,brushing teeth is good *******.,Madhu
4,remember,3_science_LS1,see hear think b ex remember and do many other,See hear think b ex ******** and do many other,"The outgoing members , whose four-year terms w...","The outgoing members , whose four-year terms w...",Remember to wash your hands.,******** to wash your hands.,Madhu
...,...,...,...,...,...,...,...,...,...
814,broken,3_science_LS1,complex substances have to be broken down into...,Complex substances have to be ****** down into...,Just how many sub secrets were being handed ov...,Just how many sub secrets were being handed ov...,The vase is broken now.,The vase is ****** now.,Madhu
815,decrease,3_science_LS1,ihe the reasons for the decrease in the poputa...,Ihe the reasons for the ******** in the poputa...,"The volume of ADC cases will decrease , Martin...","The volume of adc cases will ******** , martin...",The temperature will decrease tonight.,The temperature will ******** tonight.,Madhu
816,roughage,3_science_LS1,corract option raw sabet about roughage compre...,Corract option raw sabet about ******** compre...,No example sentence found.,No example sentence found.,Eating roughage is good for digestion.,Eating ******** is good for digestion.,Madhu
817,over,3_science_LS1,horse we used coloured cloth over the framewor...,Horse we used coloured cloth **** the framewor...,It permits the state to take over bank account...,It permits the state to take **** bank account...,The game is over.,The game is ****.,Madhu


In [35]:
#updating the word list and word list all
df1 = pd.read_csv("word_list.csv")
df2 =  pd.read_csv(file_name+"_11_word_list.csv")
df2['user'] = user
columns=['user', 'word', 'word_type', 'chunked_sentence', 'masked_chunked_sentence', 'example_sentence', 'masked_example_sentence', 'openai_sentence', 'masked_openai_sentence']
df2=df2[columns]
df_appended = pd.concat([df1, df2])
df_appended = df_appended.drop_duplicates()
df_appended.to_csv("word_list.csv", index=False)
df_appended


,user,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,user,about,Sight Words,NaN,NaN,The Highway Department source told The Constit...,The highway department source told the constit...,This book is about animals.,This book is ***** animals.
1,user,after,Sight Words,NaN,NaN,"He will be succeeded by Ivan Allen Jr. , who b...","He will be succeeded by ivan allen jr. , who b...",Lunch is after class.,Lunch is ***** class.
2,user,all,Sight Words,NaN,NaN,`` This is one of the major items in the Fulto...,`` this is one of the major items in the fulto...,All the cookies are gone.,*** the cookies are gone.
3,user,am,Sight Words,NaN,NaN,`` But I am not in favor of a sales or state i...,`` but i ** not in favor of a sales or state i...,I am happy today.,I ** happy today.
4,user,an,Sight Words,NaN,NaN,The Fulton County Grand Jury said Friday an in...,The fulton county gr**d jury said friday ** in...,I saw an elephant.,I saw ** eleph**t.
...,...,...,...,...,...,...,...,...,...
5670,Madhu,broken,3_science_LS1,complex substances have to be broken down into...,Complex substances have to be ****** down into...,Just how many sub secrets were being handed ov...,Just how many sub secrets were being handed ov...,The vase is broken now.,The vase is ****** now.
5671,Madhu,decrease,3_science_LS1,ihe the reasons for the decrease in the poputa...,Ihe the reasons for the ******** in the poputa...,"The volume of ADC cases will decrease , Martin...","The volume of adc cases will ******** , martin...",The temperature will decrease tonight.,The temperature will ******** tonight.
5672,Madhu,roughage,3_science_LS1,corract option raw sabet about roughage compre...,Corract option raw sabet about ******** compre...,No example sentence found.,No example sentence found.,Eating roughage is good for digestion.,Eating ******** is good for digestion.
5673,Madhu,over,3_science_LS1,horse we used coloured cloth over the framewor...,Horse we used coloured cloth **** the framewor...,It permits the state to take over bank account...,It permits the state to take **** bank account...,The game is over.,The game is ****.


In [34]:
#updating the word list and word list all
df1 = pd.read_csv("word_list_all.csv")
df2 =  pd.read_csv(file_name+"_11_word_list.csv")
df2['user'] = user
columns=['user', 'word', 'word_type', 'chunked_sentence', 'masked_chunked_sentence', 'example_sentence', 'masked_example_sentence', 'openai_sentence', 'masked_openai_sentence']
df2=df2[columns]
df_appended = pd.concat([df1, df2])
df_appended = df_appended.drop_duplicates()
df_appended.to_csv("word_list_all.csv", index=False)
df_appended

,user,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,user,about,Sight Words,NaN,NaN,The Highway Department source told The Constit...,The highway department source told the constit...,This book is about animals.,This book is ***** animals.
1,user,after,Sight Words,NaN,NaN,"He will be succeeded by Ivan Allen Jr. , who b...","He will be succeeded by ivan allen jr. , who b...",Lunch is after class.,Lunch is ***** class.
2,user,all,Sight Words,NaN,NaN,`` This is one of the major items in the Fulto...,`` this is one of the major items in the fulto...,All the cookies are gone.,*** the cookies are gone.
3,user,am,Sight Words,NaN,NaN,`` But I am not in favor of a sales or state i...,`` but i ** not in favor of a sales or state i...,I am happy today.,I ** happy today.
4,user,an,Sight Words,NaN,NaN,The Fulton County Grand Jury said Friday an in...,The fulton county gr**d jury said friday ** in...,I saw an elephant.,I saw ** eleph**t.
...,...,...,...,...,...,...,...,...,...
5670,Madhu,broken,3_science_LS1,complex substances have to be broken down into...,Complex substances have to be ****** down into...,Just how many sub secrets were being handed ov...,Just how many sub secrets were being handed ov...,The vase is broken now.,The vase is ****** now.
5671,Madhu,decrease,3_science_LS1,ihe the reasons for the decrease in the poputa...,Ihe the reasons for the ******** in the poputa...,"The volume of ADC cases will decrease , Martin...","The volume of adc cases will ******** , martin...",The temperature will decrease tonight.,The temperature will ******** tonight.
5672,Madhu,roughage,3_science_LS1,corract option raw sabet about roughage compre...,Corract option raw sabet about ******** compre...,No example sentence found.,No example sentence found.,Eating roughage is good for digestion.,Eating ******** is good for digestion.
5673,Madhu,over,3_science_LS1,horse we used coloured cloth over the framewor...,Horse we used coloured cloth **** the framewor...,It permits the state to take over bank account...,It permits the state to take **** bank account...,The game is over.,The game is ****.
